In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from fake_useragent import UserAgent
from datetime import datetime
import time


In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [ ]:
def SMTVsports(smtv_schedule):
    
    result = []
    for i in range(len(smtv_schedule["result"])):

        for j in range(len(smtv_schedule["result"][i]["data"])):

            smtv_type = smtv_schedule["result"][i]["data"][j]["live_types_name"]
            smtv_type_child = smtv_schedule["result"][i]["data"][j]["live_types_child_name"]
            smtv_lnameEN = smtv_schedule["result"][i]["data"][j]["lname"]
            smtv_time = smtv_schedule["result"][i]["data"][j]["time"]
            smtv_TeamA = smtv_schedule["result"][i]["data"][j]["hname"]
            smtv_TeamB = smtv_schedule["result"][i]["data"][j]["aname"]
            smtv_signal = []

            for k in smtv_schedule["result"][i]["data"][j]["account"]:
                smtv_signal.append(k["nickname"])

            dict_={
                "smtv_type":smtv_type,
                "smtv_type_child":smtv_type_child,
                "smtv_lnameEN":smtv_lnameEN,
                "smtv_time":smtv_time,
                "smtv_TeamA":smtv_TeamA,
                "smtv_TeamB":smtv_TeamB,
                "smtv_signal":smtv_signal
            }
            result.append(dict_)
            
    return result
        
    

In [ ]:
url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date=2020-06-01&now=Y&filter_live=Y"
res = requests.get(url,headers=headers)
smtv_schedule = json.loads(res.text)
smtv = SMTVsports(smtv_schedule)

In [ ]:
url_2 = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date=2020-06-01&now=Y&filter_live=Y"
res_2 = requests.get(url_2,headers=headers)
smzb_schedule = json.loads(res_2.text)
smzb = SMTVsports(smzb_schedule)

In [ ]:
smtv[0]

In [ ]:
# 平台 賽程時間 賽事 type A隊伍 B隊伍 信號[]

In [ ]:
# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

#------------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

#------------------------------

# ggzb 電競
# https://ggzb.io/

#------------------------------

# lj88 體育
# https://lj88.tv/

#------------------------------

# hqzb 體育
# https://hqzb.tv/program

#------------------------------

In [ ]:
# https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date=2020-05-31&now=Y&filter_live=Y
# https://smzb.io/program/listwithgame/default?parent=2&child=all&day_length=7&date=2020-05-31&now=Y&filter_live=Y
# https://smzb.io/program/listwithgame/default?parent=45&child=all&day_length=7&date=2020-05-31&now=Y&filter_live=Y
# https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date=2020-06-05&now=Y&filter_live=N
# https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date=2020-06-05&now=Y&filter_live=Y

In [ ]:
url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date=2020-06-01&now=Y&filter_live=Y"
res = requests.get(url,headers=headers)
smtv_schedule = json.loads(res.text)

result = []
for i in range(len(smtv_schedule["result"])):

    for j in range(len(smtv_schedule["result"][i]["data"])):

        smtv_type = smtv_schedule["result"][i]["data"][j]["live_types_name"]
        smtv_type_child = smtv_schedule["result"][i]["data"][j]["live_types_child_name"]
        smtv_lnameEN = smtv_schedule["result"][i]["data"][j]["lname"]
        smtv_time = smtv_schedule["result"][i]["data"][j]["time"]
        smtv_TeamA = smtv_schedule["result"][i]["data"][j]["hname"]
        smtv_TeamB = smtv_schedule["result"][i]["data"][j]["aname"]
        smtv_signal = []

        for k in smtv_schedule["result"][i]["data"][j]["account"]:
            smtv_signal.append(k["nickname"])

        dict_={
            "smtv_type":smtv_type,
            "smtv_type_child":smtv_type_child,
            "smtv_lnameEN":smtv_lnameEN,
            "smtv_time":smtv_time,
            "smtv_TeamA":smtv_TeamA,
            "smtv_TeamB":smtv_TeamB,
            "smtv_signal":smtv_signal
        }
        result.append(dict_)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from fake_useragent import UserAgent
from datetime import datetime
import time

ua = UserAgent()
headers = {"User-Agent":ua.random}

def GetSchedule(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            dict_={
                "live_types_name":live_types_name,
                "live_types_child_name":live_types_child_name,
                "lname":lname,
                "time":time,
                "hname":hname,
                "aname":aname,
                "account":account
            }
            result.append(dict_)

    df = pd.DataFrame(result)

    return df


smtv_url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date=2020-06-05&now=Y&filter_live=Y"
smtv_df = GetSchedule(url=smtv_url)

# smzb_url = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date=2020-06-05&now=Y&filter_live=Y"
# smzb_df = GetSchedule(url=smzb_url)

print(smtv_df)
# print(smzb_df)


